In [2]:
import gym
from gym import spaces
from gym.utils import seeding
import or_gym
from or_gym.envs.classic_or.knapsack import BoundedKnapsackEnv, KnapsackEnv
import numpy as np

In [3]:
env = gym.make('Knapsack-v1')

In [23]:
class OnlineKnapsackEnv(BoundedKnapsackEnv):
    
    def __init__(self):
        BoundedKnapsackEnv.__init__(self)
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(self.N),
            spaces.Discrete(self.N),
            spaces.Discrete(3)))

        self.step_counter = 0
        self.step_limit = 5
        
        self.seed()
        self.state = self.reset()
        
    def step(self, action):
        # Check that item will fit
        if bool(action):
            # Check that item will fit
            if self.item_weights[self.current_item] + self.current_weight <= self.max_weight:
                self.current_weight += self.item_weights[self.current_item]
                reward = self.item_values[self.current_item]
                if self.current_weight == self.max_weight:
                    done = True
                else:
                    done = False
            else:
                # End if over weight
                reward = 0
                done = True
        else:
            reward = 0
            done = False
            
        self.step_counter += 1
        if self.step_counter >= self.step_limit:
            done = True
            
        self._update_state()
        return self.state, reward, done, {}
    
    def _update_state(self):
        self.current_item = np.random.choice(self.item_numbers, p=self.item_probs)
        self.state = (
            self.item_weights,
            self.item_values,
            self.item_probs,
            self.max_weight,
            np.array([
                self.current_weight,
                self.current_item
            ]))
        
    def sample_action(self):
        return np.random.choice([0, 1])
    
    def reset(self):
        self.item_probs = self.item_limits_init / self.item_limits_init.sum()
        self.current_weight = 0
        self.item_limits = self.item_limits_init.copy()
        self._update_state()
        return self.state